In [ ]:
# rag_system.ipynb
# Asumimos que las variables de data_processing.ipynb (df, preprocess_text) y imports.ipynb (TfidfVectorizer, cosine_similarity) están disponibles.

rag_vectorizer = None
response_vectors = None

# Asegúrate de que 'df' esté cargado y 'Respuesta_Processed' exista
if df is not None and 'Respuesta_Processed' in df.columns:
    # Crear un vectorizador TF-IDF para las respuestas (la "base de conocimiento")
    # En producción, solo cargarías este vectorizador y los response_vectors
    rag_vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words=None)
    response_vectors = rag_vectorizer.fit_transform(df['Respuesta_Processed'])
    print("Vectorizador RAG y vectores de respuesta creados (o listos para cargar).")
else:
    print("No se pudo configurar RAG debido a un error de carga de datos o columna faltante.")


# Función para recuperar información relevante
def retrieve_info(query, top_n=3):
    if rag_vectorizer is None or response_vectors is None:
         return [{"Respuesta_Relevante": "Error: Sistema de recuperación no configurado.", "Similitud": 0.0}]

    processed_query = preprocess_text(query)
    query_vector = rag_vectorizer.transform([processed_query])

    # Calcular similitud coseno
    similarities = cosine_similarity(query_vector, response_vectors).flatten()

    # Obtener los índices de las respuestas más similares
    # Asegurarse de que top_n no sea mayor que el número de respuestas disponibles
    top_indices = similarities.argsort()[-min(top_n, len(df)):][::-1]

    # Recuperar las respuestas originales
    retrieved_data = []
    for i in top_indices:
        retrieved_data.append({
            # 'Consulta_Similar': df.loc[i, 'Consulta'], # Opcional, para contexto
            'Respuesta_Relevante': df.loc[i, 'Respuesta_SAE'],
            'Similitud': float(similarities[i]) # Convertir a float para serialización JSON
        })
    return retrieved_data

# Ejemplo de uso (para probar este notebook individualmente)
# if df is not None:
#     test_query = "¿Cuál es la fecha límite para matricularme?"
#     relevant_info = retrieve_info(test_query, top_n=2)
#     print(f"\nTest Query: '{test_query}'")
#     for item in relevant_info:
#         print(f"  - Respuesta relevante: '{item['Respuesta_Relevante']}' (Similitud: {item['Similitud']:.2f})")